In [ ]:
import csv
from io import StringIO
import os
import git
import pandas as pd
import re
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
import hashlib
from sklearn.utils import resample
from matplotlib import pyplot
import shutil
import subprocess
import appscript
import time
import researchpy as rp
import sys
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder 
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

## Cloning all repos into localhost

In [ ]:
# Specifying the path to where the names of the projects are stored
csv_java_repos = pd.read_csv('path to csv file with porject names', sep='$')

path  = "path to cloning directory" 

for index, row in csv_java_repos.iterrows():
    clone = "git clone https://github.com/" + row.project + ".git" 
    os.system("sshpass -p mm6pm18s9 ssh khaledwalidsabbagh")
    os.chdir(path) # Specifying the path where the cloned project needs to be copied
    os.system(clone) # Cloning


In [ ]:
df_balanced = []
for name in repo_names: #repo_names is a list that contains all project names
    try:
        repo = git.Repo("path to directory where all cloned repos exist" + name ) 
        commits_list = list(repo.iter_commits())
        commits_repo = []
        data= []
        for commit in commits_list: 
             commits_repo.append(commit.hexsha)
        
        #names of the metrics that we plan to extract
        col_list = ["git_diff_src_churn", "gh_diff_files_modified",
                    "gh_sloc", "gh_num_commits_on_files_touched", "gh_num_commits_in_push", "tr_original_commit",
                    "gh_team_size", "git_prev_commit_resolution_status", "gh_num_commit_comments",
                    "git_num_all_built_commits", "gh_diff_files_added", "gh_diff_files_deleted", "gh_build_started_at",
                    "gh_diff_src_files", "gh_diff_doc_files", "gh_diff_other_files", "tr_status"]
        
        #loading data with the desired metrics only
        cod_info = pd.read_csv('path to TravisTorrent data-set' + name+'.csv', sep=',', usecols=col_list)
        cod_info = cod_info.sort_values(by ='gh_build_started_at', ascending=True)
        
        
        count_not_found = 0
        count_match = 0 
        verdict = 0
        
        #now we define the format of the header that ccflex accepts
        header = ["id","line","contents","class_name","class_value","path","date"] 

        #now we start by writing the header in a csv file
        with open('path to where the csv should be written', 'a', newline='', encoding="utf-8") as file:
                                    writer = csv.writer(file, quoting=csv.QUOTE_ALL, delimiter='$')
                                    writer.writerow([g for g in header])
        
        #now we start computing diff between pairs of hash commits and do some filtering of lines using RegExp
        for count, commit_CIBench in enumerate(cod_info.tr_original_commit):
                 if (str(commit_CIBench) in commits_repo and count<len(cod_info)):
                    source_commit = repo.commit(str(commit_CIBench))
                    if (count+ 1 < len(cod_info)):
                        if (cod_info.tr_original_commit.loc[count+1] in commits_repo):
                                target_commit = repo.commit(str(cod_info.tr_original_commit.loc[count+1]))
                                git_diff= repo.git.diff(source_commit, target_commit)
                                pattern_code = re.compile('([+].*\n)|([+]\s.*\n)|([+++].*\n)|([+][a-zA-Z]+.*)')
                                code_result = re.findall(pattern_code, git_diff)
                                code_to_str= ''.join(''.join(elems) for elems in code_result)
                                if cod_info.tr_status[count] == "passed": 
                                      verdict = 1
                                elif cod_info.tr_status[count] == "failed": 
                                      verdict = 0
                                        
                                writeDiff(code_to_str, verdict, cod_info.gh_build_started_at.loc[count+1], name)
                                count_match = count_match + 1
                        else:
                            count_not_found = count_not_found + 1
                    else: 
                        break
        print ("number of commits in travistorrent file: " + str(count_match))
        print ("number of commits not found in repo: " + str(count_not_found))
        
        #now we write the extracted diff of lines of code
        _lines = pd.read_csv('projects/train-lines/diff_'+ name + '.csv', sep='$')
        WriteBuildInformation(name, _lines, False)

        df_majority = _lines[_lines.class_value == 1] 
        df_minority = _lines[_lines.class_value == 0]

        #now we check the distribution of lines of code and see if we should resample the minority class
        if BalancingType(df_majority, df_minority) == -1:
            df_balanced = Balance_Classes(df_majority,df_minority, -1)
        elif BalancingType(df_majority, df_minority) == 1:
            df_balanced = Balance_Classes(df_majority,df_minority, 1)
        else:
            df_balanced = _lines
        
        #after resampling, we will write a new csv version of all extracted lines of code
        df_balanced.to_csv('path to where you want to save the code changes in a csv file', sep='$', quotechar='"', quoting=csv.QUOTE_ALL, index=False)
        
        #now we need move the csv file of code changes into the ccflex directory. This is important as ccflex
        #is programmed to read input files from a specific directory pyccflex/ccflex_tmp/processing/
        
        shutil.copy('path to csv file', 'path to ccflex directory/pyccflex/ccflex_tmp/processing/')
        os.rename('path to csv file', 'path to ccflex directory/train-lines.csv')

        WriteBuildInformation(name, df_balanced, False)
        
        # we now call ccflex to start generating feature vectors from the extracted code changes
        Create_BoW(name)
        _bow = pd.read_csv('train-bag-of-words-'+ name + '.csv', sep='$')
        #  we now need to generate experimental subjects to run the experiment on
        Stratify_Sample_BoW(_bow, name)
        #after splitting the feature vectors into 10 folds, we start by training a model on every combination of 
        #9 folds
        Learn_Build_Prediction(name, False)
        
        #now we repeat the same procedure for the build records without the need to generate feature vectors
        #----------------------------------------------------------
        df_majority = cod_info[cod_info.tr_status=='passed'] 
        df_minority = cod_info[cod_info.tr_status=='failed']
        
        if BalancingType(df_majority, df_minority) == -1:
            df_balanced = Balance_Classes(df_majority,df_minority, -1)
        elif BalancingType(df_majority, df_minority) == 1:
            df_balanced = Balance_Classes(df_majority,df_minority, 1)
        else:
            df_balanced = cod_info
        WriteBuildInformation(name, cod_info, True)
        Stratify_Samples(df_balanced, 'tsm', name)
        Learn_Build_Prediction(name, True)
        
    except:
        print("An exception occurred in project: " + name)  

In [ ]:
def WriteBuildInformation(projectname, _builds, isBuild):
    if isBuild:
        filepath = 'projects/buildinfo.csv'
        build_header = ["Project Name", "Total Builds", "Failing Builds", "Passing Builds"]
    else:
        filepath = 'projects/lineinfo.csv'
        build_header = ["Project Name", "Total Analyzed Lines", "Failing Lines", "Passing Lines"]

        
    with open(filepath, mode = 'a', newline='', encoding="utf-8") as file:
                writer = csv.DictWriter(file, quoting=csv.QUOTE_ALL, delimiter='$', fieldnames = build_header)
                _builds = _builds.dropna()
                if isBuild == True:
                        
                        writer.writerow({'Project Name': projectname, 
                                         'Total Builds': len(_builds), 
                                         'Failing Builds': len(_builds[_builds.tr_status=='failed']), 
                                         'Passing Builds': len(_builds[_builds.tr_status=='passed'])
                                        }
                                       )   
                       
                else: 
                        writer.writerow({'Project Name':  projectname, 
                                         'Total Analyzed Lines': len(_builds), 
                                         'Failing Lines': len(_builds[_builds.class_value==0]), 
                                         'Passing Lines': len(_builds[_builds.class_value==1])
                                        }
                                       )

In [ ]:
def BalancingType(df_majority, df_minority):
    if (len(df_majority.index) + len(df_minority.index)) != 0:
        if (len(df_majority.index))/(len(df_majority.index) + len(df_minority.index)) > 0.6: 
            return -1

        elif (len(df_minority.index))/(len(df_majority.index) + len(df_minority.index)) > 0.6:
            return 1
        else:
            return 0        

In [ ]:
def Balance_Classes(df_majority, df_minority, action): #an action of -1 means that the minority class should be upsampled
    if action == -1:
        df_minority_upsampled = resample(df_minority, 
                                         replace=True,     
                                         n_samples=len(df_majority.index),
                                         random_state=123)

        print ('upsampled the minority class to ' + str(len(df_majority.index))) 
        return pd.concat([df_majority, df_minority_upsampled])

    elif action == 1: 
        df_majority_upsampled = resample(df_majority, 
                                         replace=True,     
                                         n_samples=len(df_minority.index),
                                         random_state=123)
        print ('upsampled the minority class to ' + str(len(df_minority.index))) 
        return pd.concat([df_minority, df_majority_upsampled])
    else:
        return

In [ ]:
def Create_BoW(repo_name):
    appscript.app('Terminal').do_script('cd path to ccflex script pyccflex/py-ccflex/; ./run2.sh') 

    #sleep for 5 seconds while ccflex completes the generation of feature vectors for one file
    while not os.path.exists('/pyccflex/ccflex_tmp/processing/train-bag-of-words.csv'):
        time.sleep(5)

    #making sure that ccflex does not overwrite existing file by renaming the file that it will generate
    if os.path.isfile('pyccflex/ccflex_tmp/processing/train-bag-of-words.csv'):
        os.rename('pyccflex/ccflex_tmp/processing/train-bag-of-words.csv', 'pyccflex/ccflex_tmp/processing/train-bag-of-words-' + repo_name + '.csv')
     

In [ ]:
def Stratify_Samples(data, typeDf, project):
    skf = StratifiedKFold(n_splits=10)
    
    if typeDf == 'tsm':
        data.loc[data.git_prev_commit_resolution_status == "merge_found", "git_prev_commit_resolution_status"] = "0"
        data.loc[data.git_prev_commit_resolution_status == "build_found", "git_prev_commit_resolution_status"] = "1"
        data.loc[data.git_prev_commit_resolution_status == "no_previous_build", "git_prev_commit_resolution_status"] = "0"
        data.loc[data.tr_status == "passed", "tr_status"] = '1'
        data.loc[data.tr_status == "failed", "tr_status"] = '0'
        data = data.dropna()
        skf.get_n_splits(data.iloc[:, :-2], data.tr_status)

        #if directory does not exist, create it
        if not os.path.isdir('subjects/'):
                os.mkdir('subjects/')
        
        #creating directories for training and testing for both TSM and TF
        os.mkdir('/subjects/' + project)
        os.mkdir('/subjects/' + project+ '/tsm')
        os.mkdir('/subjects/' + project+ '/tsm/training/')
        os.mkdir('/subjects/' + project+ '/tsm/testing/')
        os.mkdir('/subjects/' + project+ '/bow')
        os.mkdir('/subjects/' + project+ '/bow/training/')
        os.mkdir('/subjects/' + project+ '/bow/testing/')
        
        counter = 0
        #splitting 
        for train_index, test_index in skf.split(data.iloc[:, :-2], data.tr_status):

            X_train, X_test, y_train, y_test = data.iloc[train_index], data.iloc[test_index], data.tr_status.iloc[train_index], data.tr_status.iloc[test_index]

            X_train.to_csv(r'subjects/' + project+ '/tsm/training/' + 'x_train_lines' + str(counter) + ".csv", sep='$', quotechar='"', quoting=csv.QUOTE_ALL, index=False)
            X_test.to_csv(r'subjects/' + project + '/tsm/testing/' + 'x_test_lines' + str(counter) + ".csv", sep='$', quotechar='"', quoting=csv.QUOTE_ALL, index=False)
            y_train.to_csv(r'subjects/' + project+ '/tsm/training/' + 'x_train_dep' + str(counter) + ".csv", sep='$', quotechar='"', quoting=csv.QUOTE_ALL, index=False)
            y_test.to_csv(r'subjects/' + project + '/tsm/testing/' + 'x_test_dep' + str(counter) + ".csv", sep='$', quotechar='"', quoting=csv.QUOTE_ALL, index=False)

            counter = counter + 1
    elif typeDf == 'bow':      
        data = data.dropna()
        skf.get_n_splits(data, data.class_value)
        counter = 0
        for train_index, test_index in skf.split(data, data.class_value):
            X_train, X_test, y_train, y_test = data.iloc[train_index], data.iloc[test_index], data.class_value.iloc[train_index], data.class_value.iloc[test_index]
            X_train.to_csv(r'subjects/' + project+ '/bow/training/' + 'x_train_lines' + str(counter) + ".csv", sep='$', quotechar='"', quoting=csv.QUOTE_ALL, index=False)
            X_test.to_csv(r'subjects/' + project + '/bow/testing/' + 'x_test_lines' + str(counter) + ".csv", sep='$', quotechar='"', quoting=csv.QUOTE_ALL, index=False)
            y_train.to_csv(r'subjects/' + project+ '/bow/training/' + 'x_train_dep' + str(counter) + ".csv", sep='$', quotechar='"', quoting=csv.QUOTE_ALL, index=False)
            y_test.to_csv(r'subjects/' + project + '/bow/testing/' + 'x_test_dep' + str(counter) + ".csv", sep='$', quotechar='"', quoting=csv.QUOTE_ALL, index=False)
            counter = counter + 1

    print ('finished splitting majority')    

In [ ]:
def Learn_Build_Prediction(repo_name, isBuild):
    counter = 0
    total_fscore = 0
    total_mcc = 0
    result_ = {}
    result_all = {}
    result_bow = {}
    forest= RandomForestClassifier(n_estimators=100, random_state=123)

    while counter < 10:
        if isBuild:
            #filepath should be set to the path where the prediction results of the TSM validation will be written 
            filepath = '/projects/tsm_results.csv'
            train_indep = pd.read_csv(r'/subjects/' + repo_name + '/tsm/training/x_train_lines' + str(counter) + '.csv', sep='$')
            train_dep = pd.read_csv(r'/subjects/' + repo_name + '/tsm/training/x_train_dep' + str(counter) + '.csv', sep='$')
            test_indep = pd.read_csv(r'/subjects/' + repo_name + '/tsm/testing/x_test_lines' + str(counter) + '.csv', sep='$')
            test_dep = pd.read_csv(r'/subjects/' + repo_name + '/tsm/testing/x_test_dep' + str(counter) + '.csv', sep='$')
            
            with open (filepath, 'a') as outcsv:
                writer = csv.writer(outcsv)
                writer.writerow(['project', 'metric', 'fold', 'precision', 'recall', 'test_f1','test_mcc'])
            for column in train_indep:
                if is_numeric_dtype(train_indep[column]):
                    forest.fit(train_indep[[column]], train_dep)   
                    y_pred = forest.predict(test_indep[[column]]) 

                    result_['project'] = repo_name
                    result_['metric'] =  column
                    result_['fold'] =  counter
                    result_['precision'] = precision_score(test_dep, y_pred)
                    result_['recall'] = recall_score(test_dep, y_pred)

                    result_['test_f1'] = f1_score(test_dep, y_pred)
                    result_['test_mcc'] = matthews_corrcoef(test_dep, y_pred)

                    total_prec = total_fscore + result_['precision']
                    total_recall = total_fscore + result_['recall']

                    total_fscore = total_fscore + result_['test_f1']
                    total_mcc = total_mcc + result_['test_mcc']

                    with open(filepath, 'a') as f:
                        fieldnames = ['project', 'metric', 'fold', 'precision', 'recall', 'test_f1','test_mcc']
                        writer = csv.DictWriter(f, fieldnames=fieldnames)
                        writer.writerow(result_)
                else:
                    continue
            new_train_indep = train_indep.select_dtypes(include=numerics)
            new_test_indep = test_indep.select_dtypes(include=numerics)

            forest.fit(new_train_indep.loc[:, new_train_indep.columns != 'tr_status'], train_dep)   
            y_pred = forest.predict(new_test_indep.loc[:, new_train_indep.columns != 'tr_status'])
            result_all['project'] = repo_name
            result_all['metric'] =  'all tsm'
            result_all['fold'] =  counter
            result_all['precision'] = precision_score(test_dep, y_pred)
            result_all['recall'] = recall_score(test_dep, y_pred)

            result_all['test_f1'] = f1_score(test_dep, y_pred)
            result_all['test_mcc'] = matthews_corrcoef(test_dep, y_pred)

            with open(filepath, 'a') as f:
                        fieldnames = ['project', 'metric', 'fold', 'precision', 'recall', 'test_f1','test_mcc']
                        writer = csv.DictWriter(f, fieldnames=fieldnames)
                        writer.writerow(result_all)
                        
        else:
            #filepath should be set to the path where the prediction results of the TF validation will be written 

            filepath = 'projects/bow_results.csv'
            train_indep = pd.read_csv(r'subjects/' + repo_name + '/bow/training/x_train_lines' + str(counter) + '.csv', sep='$')
            train_dep = pd.read_csv(r'subjects/' + repo_name + '/bow/training/x_train_dep' + str(counter) + '.csv', sep='$')
            test_indep = pd.read_csv(r'subjects/' + repo_name + '/bow/testing/x_test_lines' + str(counter) + '.csv', sep='$')
            test_dep = pd.read_csv(r'subjects/' + repo_name + '/bow/testing/x_test_dep' + str(counter) + '.csv', sep='$')
            
            with open (filepath, 'a') as outcsv:
                writer = csv.writer(outcsv)
                writer.writerow(['project', 'metric', 'fold', 'precision', 'recall', 'test_f1','test_mcc'])
                
            new_train_indep = train_indep.select_dtypes(include=numerics)
            new_test_indep = test_indep.select_dtypes(include=numerics)

            forest.fit(new_train_indep.loc[:, new_train_indep.columns != 'class_value'], train_dep)   
            y_pred = forest.predict(new_test_indep.loc[:, new_train_indep.columns != 'class_value'])
            result_bow['project'] = repo_name
            result_bow['metric'] =  'token counts'
            result_bow['fold'] =  counter
            result_bow['precision'] = precision_score(test_dep, y_pred)
            result_bow['recall'] = recall_score(test_dep, y_pred)

            result_bow['test_f1'] = f1_score(test_dep, y_pred)
            result_bow['test_mcc'] = matthews_corrcoef(test_dep, y_pred)

            with open(filepath, 'a') as f:
                        fieldnames = ['project', 'metric', 'fold', 'precision', 'recall', 'test_f1','test_mcc']
                        writer = csv.DictWriter(f, fieldnames=fieldnames)
                        writer.writerow(result_bow)

        counter = counter + 1
        print('counter: ' + str(counter))